# DoE with classical designs

This is a template notebook for classical designs of experiments.

Author: {{ cookiecutter.author_name }}
Created: {{ cookiecutter.timestamp }}

## How to use the notebook

The following cells:
- specify the parameter search space, the objective, and any other metrics,
- specifiy, which classical design should be employed, and
- computes the list of suggested parameter sets, with which to run trials in your experiment. 

By default, the notebook is set up to run with an example. To see how it works, run the notebook (multiple times) without changing the code.

For your project, adjust the code in the linked cells with your objectives, variables, dataset etc. and then execute all cells in order.

Please refer to classical_doe.board for detailed instructions.

In [0]:
# Link to project experiments folder hypothesis_experiment_learnings.board (refresh and hit enter on this line to see the link)

In [0]:
# <halerium id="3000f30b-1f20-4035-8ef0-218a1a070dc1">
# Link to classical_doe.board
# </halerium id="3000f30b-1f20-4035-8ef0-218a1a070dc1">


## Imports  and general setup

In [0]:
import numpy as np
import pandas as pd

### 2. Specify the Data Path

In [0]:
experiment_name = 'classical_doe'  # please provide a name for the experiment
# <halerium id="dc8be860-579e-40fe-b3f6-024589b3c6ef">
data_dir = "./"           # please provide a name for saving the trial data for the experiment
# </halerium id="dc8be860-579e-40fe-b3f6-024589b3c6ef">


### 3. Specify the Metrics

In [0]:
# <halerium id="e42d5eb3-07d1-493b-9430-555a397f22c0">
metrics = ["cost", "quality"]   # please provide a list of metrics
# </halerium id="e42d5eb3-07d1-493b-9430-555a397f22c0">


### Specify the Parameters

In [0]:
# <halerium id="2e752258-a41d-4508-b092-e7103d64478a">
parameters = [
    # please insert the information on the names and bound/values of the parameters to try:
    {
        "name": "x1",           # the name of the parameter
        "type": "range",        # the type of parameter: "range" is for continuous parameters
        "bounds": [0., 1.],     # the lower and upper bound of the parameter as a tuple for range parameters
        "n_values": 3,          # the number of different values for the range parameter to try
    },
    {
        "name": "x2",
        "type": "range",
        "bounds": [0., 10.],
        "n_values": 3, 
    },  
    {
        "name": "x3",
        "type": "range",
        "bounds": [-5., 5.],
        "n_values": 3, 
    },  
#    {
#        "name": "x4",
#        "type": "choice",                    # the type of parameter: "choice" is for discrete parameters
#        "values": ["up", "down", "stange"],  # the values to try from for parameter#
#        "is_ordered": False,                 # whether values are ordered
#    },   
]
# </halerium id="2e752258-a41d-4508-b092-e7103d64478a">


### 5. Choose the Design

In [0]:
# <halerium id="fff941c8-82f9-40d5-8b5e-76b9f56b4211">
design_type = "full_factorial"               # the type of design, available are:
                                             #  - "simple_central": just a central design
                                             #  - "mixed_central": central + mixed second order design
                                             #  - "full_central": central + full 2-level factorial design
                                             #  - "full_factorial": full factorial design
                                             #  - "random_factorial": random factorial design
                                             #  - "random": random design

n_trials = 100                               # number of trials unless determined by design
sort_values = False                          # whether to sort trials by parameter values
sort_ascending = True                        # whether to sort in ascending order
# </halerium id="fff941c8-82f9-40d5-8b5e-76b9f56b4211">


## Expected performance for polynomial models

In case, all parameters are continuous, you can compute the expected performance of the design for estimating the parameters of a polynomial model.

In [0]:
# <halerium id="fff941c8-82f9-40d5-8b5e-76b9f56b4211">
order = 2                                   # the order of the polynomial model
mixed = False                               # whether to include mixed terms for order >= 2
# </halerium id="fff941c8-82f9-40d5-8b5e-76b9f56b4211">


### 6. Run the Trials

#### Get the design

In [0]:
from functions.classical_doe import get_doe_design

# <halerium id="5fc2585e-3bed-4036-80fd-4a4d9a41014a">
design = get_doe_design(metrics, parameters, design_type, n_trials, sort_values, sort_ascending, data_dir, experiment_name)
# </halerium id="5fc2585e-3bed-4036-80fd-4a4d9a41014a">


### 7. Judge the design

In [0]:
from functions.classical_doe import judge_design

# <halerium id="e6b5ea26-d56b-42d9-8c51-42469967c149">
judge_design(design, design_type, mixed, order, parameters, metrics)
# </halerium id="e6b5ea26-d56b-42d9-8c51-42469967c149">
